In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from keras.preprocessing.text import text_to_word_sequence
from keras.preprocessing.text import Tokenizer  
from keras.preprocessing.sequence import pad_sequences
from keras import models
from keras import layers
from keras import losses
from keras import metrics
from keras import optimizers
from keras.utils import plot_model

import numpy as np
import matplotlib.pyplot as plt

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

from collections import Counter
from pathlib import Path
import os
import numpy as np
import re
import string
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
from nltk.corpus import wordnet
import unicodedata
import html
stop_words = stopwords.words('english')
from keras.callbacks import EarlyStopping

In [ ]:
Data= pd.read_csv('/kaggle/input/jigsaw-unintended-bias-in-toxicity-classification/train.csv')

In [ ]:
Data=Data.iloc[0:100000,:]

In [ ]:
x_Data= Data['comment_text']
y_Data= Data['target']

In [ ]:
del Data

In [ ]:
train_data, test_data, train_labels, test_labels = train_test_split(x_Data, 
                                                                    y_Data,
                                                                    test_size=0.33,
                                                                    random_state=42)

In [ ]:
Unprocessed_train_data= train_data.to_list()
Unprocessed_test_data= test_data.to_list()
#Unprocessed_train_labels= train_labels.to_list()
#Unprocessed_test_labels= test_labels.to_list()

In [ ]:
del train_data
del test_data
del x_Data
del y_Data

## Text Pre-Processing

In [ ]:
def remove_special_chars(text):
    re1 = re.compile(r'  +')
    x1 = text.lower().replace('#39;', "'").replace('amp;', '&').replace('#146;', "'").replace(
        'nbsp;', ' ').replace('#36;', '$').replace('\\n', "\n").replace('quot;', "'").replace(
        '<br />', "\n").replace('\\"', '"').replace('<unk>', 'u_n').replace(' @.@ ', '.').replace(
        ' @-@ ', '-').replace('\\', ' \\ ')
    return re1.sub(' ', html.unescape(x1))


def remove_non_ascii(text):
    """Remove non-ASCII characters from list of tokenized words"""
    return unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')


def to_lowercase(text):
    return text.lower()


def remove_punctuation(text):
    """Remove punctuation from list of tokenized words"""
    translator = str.maketrans('', '', string.punctuation)
    return text.translate(translator)


def replace_numbers(text):
    """Replace all interger occurrences in list of tokenized words with textual representation"""
    return re.sub(r'\d+', ' ', str(text))


def remove_whitespaces(text):
    return text.strip()


def remove_stopwords(words, stop_words):
    """
    :param words:
    :type words:
    :param stop_words: from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS
    or
    from spacy.lang.en.stop_words import STOP_WORDS
    :type stop_words:
    :return:
    :rtype:
    """
    return [word for word in words if word not in stop_words]


def stem_words(words):
    """Stem words in text"""
    stemmer = PorterStemmer()
    return [stemmer.stem(word) for word in words]

def lemmatize_words(words):
    """Lemmatize words in text"""

    lemmatizer = WordNetLemmatizer()
    return [lemmatizer.lemmatize(word) for word in words]

def lemmatize_verbs(words):
    """Lemmatize verbs in text"""

    lemmatizer = WordNetLemmatizer()
    return ' '.join([lemmatizer.lemmatize(word, pos='v') for word in words])

def text2words(text):
  return word_tokenize(text)

def normalize_text( text):
    text = remove_special_chars(text)
    text = remove_non_ascii(text)
    text = remove_punctuation(text)
    text = to_lowercase(text)
    text = replace_numbers(text)
    words = text2words(text)
    words = remove_stopwords(words, stop_words)
    #words = stem_words(words)# Either stem ovocar lemmatize
    words = lemmatize_words(words)
    words = lemmatize_verbs(words)

    return ''.join(words)

In [ ]:
def normalize_corpus(corpus):
  return [normalize_text(t) for t in corpus]

In [ ]:
train_data= normalize_corpus(Unprocessed_train_data)
test_data= normalize_corpus(Unprocessed_test_data)

In [ ]:
import gc
gc.collect()

## BOW Model Using Binary Features:

In [ ]:
vocab_sz = 10000 # None means all
tok = Tokenizer(num_words=vocab_sz, oov_token='UNK')
tok.fit_on_texts(train_data)

In [ ]:
# Extract binary BoW features
x_train = tok.texts_to_matrix(train_data, mode='binary')

In [ ]:
del train_data
gc.collect()

In [ ]:
x_test = tok.texts_to_matrix(test_data, mode='binary')

In [ ]:
del test_data
gc.collect()

In [ ]:
y_train = np.asarray(train_labels).astype('float32')

In [ ]:
del train_labels
gc.collect()

In [ ]:
y_test = np.asarray(test_labels).astype('float32')

In [ ]:
del test_labels
gc.collect()

In [ ]:
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

In [ ]:
gc.collect()

## Model using Binary Features:

In [ ]:
from keras import regularizers
model_1 = models.Sequential()
model_1.add(layers.Dense(16, activation='relu', input_shape=(10000,)))
model_1.add(layers.Dropout(0.6))
model_1.add(layers.Dense(8, kernel_regularizer=regularizers.l2(0.02), activation='relu'))
model_1.add(layers.Dense(1, activation='sigmoid'))
model_1.summary()

In [ ]:
model_1.compile(optimizer=optimizers.RMSprop(lr=0.001),
              loss=losses.MeanSquaredError(name="mean_squared_error"),
              metrics=[metrics.MeanSquaredError(name="mean_squared_error")])

In [ ]:
model_1.fit(x_train, y_train,
                    epochs=5,
                    batch_size=8,
                    validation_data=(x_test, y_test))

In [ ]:
gc.collect()

## Using Glove pretrained embeddings:

In [ ]:
train_data= normalize_corpus(Unprocessed_train_data)
test_data= normalize_corpus(Unprocessed_test_data)

In [ ]:
import gc
gc.collect()

In [ ]:
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip glove.6B.zip

In [ ]:
vocab_sz = 10000 
tok = Tokenizer(num_words=vocab_sz, oov_token='UNK')
tok.fit_on_texts(train_data)

In [ ]:
# load the whole embedding into memory
embeddings_index = dict()
f = open('glove.6B.100d.txt', mode='rt', encoding='utf-8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()
print('Loaded %s word vectors.' % len(embeddings_index))
# create a weight matrix for words in training docs
embedding_matrix = np.zeros((vocab_sz, 100))
for word, i in tok.word_index.items():
    if i == vocab_sz:
        break
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [ ]:
# integer encode the documents
train_data_sequence =tok.texts_to_sequences(train_data)

In [ ]:
del train_data
gc.collect()

In [ ]:
test_data_sequence =tok.texts_to_sequences(test_data)

In [ ]:
del test_data
gc.collect()

In [ ]:
# pad documents to a max length of 30 words
max_length = 100
x_train_= pad_sequences(train_data_sequence, maxlen=max_length, padding='post')

In [ ]:
del train_data_sequence
gc.collect()

In [ ]:
x_test= pad_sequences(test_data_sequence, maxlen=max_length, padding='post')

In [ ]:
del test_data_sequence
gc.collect()

In [ ]:
print(x_train_.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

## LSTM Model

In [ ]:
from keras.layers import LSTM, GRU
n_latent_factors = 100
model_2 = models.Sequential()
model_2.add(layers.Embedding(vocab_sz, n_latent_factors, weights=[embedding_matrix], input_length=max_length, trainable=False))
model_2.add(LSTM(100, return_sequences=True))
model_2.add(LSTM(100))
model_2.add(layers.Dense(100, activation='relu'))
model_2.add(layers.Dense(16, activation='relu'))
model_2.add(layers.Dense(16, activation='relu'))
model_2.add(layers.Dense(1, activation='sigmoid'))
model_2.summary()

In [ ]:
model_2.compile(optimizer=optimizers.RMSprop(lr=0.0001),
              loss=losses.MeanSquaredError(name="mean_squared_error"),
              metrics=[metrics.MeanSquaredError(name="mean_squared_error")])
model_2.fit(x_train_,y_train,
                    epochs=10,
                    batch_size=64,
                    validation_data=(x_test,y_test))

## GRU Model

In [ ]:
model_3 = models.Sequential()
model_3.add(layers.Embedding(vocab_sz, n_latent_factors, weights=[embedding_matrix], input_length=max_length, trainable=False))
model_3.add(GRU(100, return_sequences=True))
model_3.add(GRU(100))
model_3.add(layers.Dense(100, activation='relu'))
model_3.add(layers.Dense(16, activation='relu'))
model_3.add(layers.Dense(16, activation='relu'))
model_3.add(layers.Dense(1, activation='sigmoid'))
model_3.summary()

In [ ]:
model_3.compile(optimizer=optimizers.RMSprop(lr=0.0001),
              loss=losses.MeanSquaredError(name="mean_squared_error"),
              metrics=[metrics.MeanSquaredError(name="mean_squared_error")])
model_3.fit(x_train_,y_train,
                    epochs=10,
                    batch_size=64,
                    validation_data=(x_test,y_test))

## Conclusion: The best performance was obtained using the GRU Model which keeps improving its performance by increasing the training epoques;
## The second best performance is the LSTM Model;